<a href="https://colab.research.google.com/github/dhsilva2912/MIT_ComputerVision/blob/main/DemoGeometry.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

if IN_COLAB:
  #copy data
  !git clone https://github.com/mbaradad/pset_geometry
  !cp -rf pset_geometry/* .
  !rm -rf pset_geometry

Cloning into 'pset_geometry'...
remote: Enumerating objects: 46, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 46 (delta 14), reused 42 (delta 10), pack-reused 0
Unpacking objects: 100% (46/46), done.


In [ ]:
''' Code loading '''
from auxiliary_functions import *
import numpy as np
import cv2
%matplotlib inline

In [ ]:
''' Data loading '''
# Load the images and the ground truth depth that we will use throughout the PSET
img_L = imread("data/000191_10_L.png")
img_R = imread("data/000191_10_R.png")
K_gt = np.load("data/000191_K.npy")

# Opencv expects the channels last, we define it here for compactness
img_L_cv2 = img_L.transpose((1,2,0))
img_R_cv2 = img_R.transpose((1,2,0))

img_height = img_R.shape[1]
img_width = img_R.shape[2]

In this Problem Set, we will study how to obtain the geometrical structure of a scene from either a pair of images (using photometric and geometric principles) or from a single image (by learning to predict the structure from single images with a Deep Convolutional Neural Network).

The data we will use for this study is that of the KITTI dataset, and the particular example example we will use throughout this notebook is displayed in the following cell. It corresponds to one of the test images of the dataset, for which the real 3D structure is not publicly available:

In [ ]:
show_image(img_L, title="Left image")
show_image(img_R, title="Right image")

To begin with, we will assume that we have access to a stereo pair of images (i.e. two images from different positions), for which we can compute how the pixels from one image relate to pixels on the other image, process which is called Stereo matching.

In the following cell, the left and right images are displayed one on top of the other in a loop. As you can see, pixels that correspond to things that are further away have less disparity ("move less"), than pixels that correspond to things that are closer to the camera, which have higher disparity ("move more").

In [ ]:
display_gif('data/both.gif')

To quantify this movement between pixels, we will use the OpenCV built-in version of the algorithm 
<a href="https://pdfs.semanticscholar.org/bcd8/4d8bd864ff903e3fe5b91bed3f2eedacc324.pdf"> Stereo processing by semiglobal matching and mutual information</a> by H. Hirschmuller. 

The following cell computes the disparity, with a set of hyperparameters that have been tuned manually to perform good in this particular instance. Once you have completed the notebook, change these parameters to see how they influence the final results. 

In [ ]:
# Based on: http://timosam.com/python_opencv_depthimage 
window_size = 3
min_disp = 1
max_disp = 1 + 16*4   # max_disp - min_disp has to be dividable by 16
left_matcher = cv2.StereoSGBM_create(
    minDisparity=min_disp,
    numDisparities=max_disp - min_disp,
    blockSize=5,
    P1=8 * 3 * window_size ** 2,
    P2=32 * 3 * window_size ** 2,
    disp12MaxDiff=1,
    uniquenessRatio=15,
    speckleWindowSize=0,
    speckleRange=2,
    preFilterCap=63,
    mode=cv2.STEREO_SGBM_MODE_SGBM_3WAY
)

right_matcher = cv2.ximgproc.createRightMatcher(left_matcher)
disparity_photometric_unfiltered = left_matcher.compute(img_L_cv2, img_R_cv2)
show_image(disparity_photometric_unfiltered, title='Unfiltered photometric disparity', normalize=True)

To further improve the disparity prediction, it is typical to compute the disparity twice by swapping the left and right images, and enforcing that both disparities are consistent. 
The following cell achieves this:

In [ ]:
lmbda = 80000
sigma = 1.2

wls_filter = cv2.ximgproc.createDisparityWLSFilter(matcher_left=left_matcher)
wls_filter.setLambda(lmbda)
wls_filter.setSigmaColor(sigma)

displ = disparity_photometric_unfiltered
dispr = right_matcher.compute(img_R_cv2, img_L_cv2)

displ = wls_filter.filter(displ, img_L_cv2, None, dispr)
disparity_photometric = np.array(displ, dtype=np.float32)/16
valid_disparity = disparity_photometric > 0

show_image(disparity_photometric, title='Filtered Photometric disparity', normalize=False, show_scale=True)

If the cameras are <a href="https://en.wikipedia.org/wiki/Image_rectification">rectified</a>, as it is the case, we can directly compute the depth (up to scale), as the inverse of the disparity as:

In [ ]:
valid_depths = 1/disparity_photometric[valid_disparity]
depth_photometric_unscaled = np.zeros_like(disparity_photometric)
depth_photometric_unscaled[valid_disparity] = valid_depths
clip_value = np.percentile(depth_photometric_unscaled.flatten(), 99)
depth_photometric_clipped_unscaled = np.clip(depth_photometric_unscaled, 0, clip_value)

show_image(depth_photometric_clipped_unscaled, title='Photometric depth unscaled', normalize=False, show_scale=True)

Finally, if we know what is the real distance between the two stereo cameras (in this case approximately 0.5335 m) and the focal length (which is stored in $K_{gt}[0][0]$ in pixel units), we can recover the real depth by considering the formula:
\begin{align}
depth = \frac{baseline \cdot focal}{disparity}
\end{align}
Which can be easily deduced using a pinhole camera model and using similar triangles: 
<img src=https://www.researchgate.net/profile/Sing_Bing_Kang/publication/2313285/figure/fig1/AS:341573188505604@1458448802555/Relationship-between-the-baseline-b-disparity-d-focal-length-f-and-depth-z.png width="50%" height="50%">


Multiplying the unscaled depth by this factor we get:

In [ ]:
depth_photometric = depth_photometric_unscaled*0.5335*K_gt[0,0]
depth_photometric_clipped = np.clip(depth_photometric, 0, 80)
show_image(depth_photometric_clipped, title='Photometric depth scaled', normalize=False, show_scale=True)

The second approach we will study is that of learning a model that is able to predict the disparity from a single image. To do this, we can either rely on training data (for which we have the real depth values) or, if we have access to pairs of images with translational motion, train the network without direct supervision, using similar principles as those of stereo matching.

One technique that implements this type of training with a CNN is <a href="https://arxiv.org/abs/1806.01260">Unsupervised Monocular Depth Estimation with Left-Right Consistency</a> by C. Godard et al., for which there is a publicly available implementation at https://github.com/mrharicot/monodepth. This method uses similar principles as <a href="https://arxiv.org/abs/1704.07813">Unsupervised Learning of Depth and Ego-Motion from Video</a> T. Zhou et al., but proposes several new techniques to improve the quality of the predictions.

For simplicity, we have already downloaded the models and computed the disparity predicted by this model. The following cell loads and displays the result for the test example:


In [ ]:
disparity_learned = np.load('precomputed_results/computed_disparity.npy')
# Resize to the same size of the original image
disparity_learned = cv2.resize(disparity_learned, (img_L.shape[2],img_L.shape[1]))
depth_learned = 1/disparity_learned
show_image(disparity_learned, title='Learned disparity', normalize=True)
clip_value = np.percentile(depth_learned.flatten(), 95)
depth_learned_clipped = np.clip(depth_learned, 0, clip_value)
show_image(depth_learned_clipped, title='Learned depth', normalize=True)

Once we have computed disparity maps in the image plane, we can produce a point cloud, by inverting the rendering equation. 
The rendering equation relates 3D world point ($x_{w}, y_{w}, z_{w}$) to image coordinates ($x_{i}$,$y_{i}$) in the image plane following:

\begin{align}
\begin{bmatrix}
       x' \\
       y' \\
       w
\end{bmatrix}&=K
\begin{bmatrix}
       x_{w} \\
       y_{w} \\
       z_{w}
\end{bmatrix};\ \ \ \ \ \ \ \ \ \ \ \
x_{i} = \frac{x'}{w}; \ \ \ \ \ \ \ \ \ \ \ \
y_{i} = \frac{y'}{w}
\end{align}

Consequently, to invert this process we require having access to the intrinsics matrix $K$, or approximate it with reasonable values. 

Assuming that the horizontal field of view of the camera is 105 degrees, and the center of projection is the center of the image, the intrinsics matrix is computed in the following cell.


In [ ]:
img_height, img_width = img_L.shape[1:3]
def approximate_K_from_horizontal_FOV(FOV_h_deg):
    FOV_h = np.deg2rad(FOV_h_deg) #...
    f_x = img_width/np.tan(FOV_h/2)#...
    f_y = img_width/np.tan(FOV_h/2) #...
    c_x = img_width/2 #...
    c_y = img_height/2 #...
    K = np.array(((f_x, 0, c_x),
                 (0, f_y, c_y),
                 (0,   0,   1)))
    return K
K = approximate_K_from_horizontal_FOV(105)

Having access to $K$, we can obtain a point cloud from a depth map by simply inverting the rendering equation. First, for each pixel, we compute its coordinates in the image plane. The previous definition of $K$ assumes that the top-left pixel has the coordinate $(0,0)$, and the right most pixel the coordinate $(width,height)$. As we do not know what $w$ corresponds to each pixel, we will assume it is $1$ for the moment, and resolve the ambiguity later using the depth. The following cell  creates the $3\times height \times width$ coordinate map.


In [ ]:
def create_image_coord_maps(height, width):
    img_coordinates_x_y = np.meshgrid(np.arange(0,width), np.arange(0,height))
    img_coordinates = np.concatenate((img_coordinates_x_y, np.ones((1, height, width))))
    return img_coordinates

show_image(create_image_coord_maps(img_height, img_width)[0], normalize=True)
show_image(create_image_coord_maps(img_height, img_width)[1], normalize=True)


In [ ]:
def backproject_depth_to_pointcloud(K, depth):
    height, width = depth.shape
    img_coordinates = create_image_coord_maps(height, width)
    
    img_coordinates_flattened = img_coordinates.reshape(3,-1)
    depth_flattened = depth.flatten()

    K_inv = np.linalg.inv(K)
    cam_coordinates = np.matmul(K_inv, img_coordinates_flattened)
    point_cloud = cam_coordinates * depth_flattened

    # Reshape to original shape
    point_cloud = point_cloud.reshape((3, *depth.shape))
    return point_cloud

#we use clipped depths for display purposes
point_cloud_learned = backproject_depth_to_pointcloud(K_gt, depth_learned_clipped)
point_cloud_photometric = backproject_depth_to_pointcloud(K_gt, depth_photometric_clipped)

In [ ]:
show_pointcloud(point_cloud_learned, img_L)

In [ ]:
show_pointcloud(point_cloud_photometric, img_L, valid_disparity, points_to_show=30000)

Finally, compare the result obtained when backprojecting using the estimated K and the ground truth K, and how the result distorts depending on its values. Try it for different values of the Field of view: for example, the horizontal Field of view of an Iphone is approximately 60 degrees.

In [ ]:
K = approximate_K_from_horizontal_FOV(105)

point_cloud_learned_estimated_K = backproject_depth_to_pointcloud(K, depth_learned_clipped)
show_pointcloud(point_cloud_learned_estimated_K, img_L, valid_disparity, points_to_show=30000)